# Inductive Model - Hops

This notebooks shows how to load an inductive model saved with pytorh, and use it to interactively evaluate unseen graphs with this model in Grasshopper through hops using the [ghhops-server-py](https://github.com/mcneel/compute.rhino3d/tree/7.x/src/ghhops-server-py) workflow.

Authors: David Leon and Dai Kandil

## **Dependencies**

This next cell installs the necesary dependencies to run the whole thing. It usually takes about 10 minutes to run ☕

!! After it has installed, remember that you will have to run it again if you let the runtime timeout after 30 minutes.

In [ ]:
!pip install ghhops-server==1.5.2 pyngrok dgl #may take few minutes (~10m)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 66.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=9dc6360054129616b0a83b279e37feb98f7301a4c198f09eef5ef8a512a42a51
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
  Created wheel for rhino3dm: filename=rhino3dm-7.14.2-cp310-cp310-linux_x86_64.whl size=4847392 sha256=6154353181acb751b307aa9cc5ea1682d3e67f19291f9a1a3da6f817e6e50d54
  Stored in directory: /root/.cache/pip/wheels/2e/43/d1/47586858aa1a6ea23f1e076755337f3f17d9ee49715a650f5b
Successfully built pyngrok rhino3dm


## Import libraries

In [ ]:
#import modules
%matplotlib inline
import numpy
import rhino3dm
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn import SAGEConv

import json
import math
import networkx as nx

import matplotlib.pyplot as plt
from flask import Flask
import ghhops_server as hs
from pyngrok import ngrok
import threading

#**Hops Workflow**


##Prepare the imported graph for prediction

We will need to preprocess the graphs we want to predict their labels, in the same way we did for the graphs we used for training the model.

##Load the trained model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_path='/content/drive/MyDrive/AI_graph/FinalSubmission/Trained_Model/Plan_Eval'

Then, define the same class that you used to train the model

In [ ]:
#model class
class SAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(SAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats,'mean')
        self.conv2 = SAGEConv(h_feats, num_classes,'mean')

    def forward(self, g, in_feat,edge_weight):  #edge_weight to include edge weight in the meassage passing, pass the weights tensor here
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata["h"] = h
        return dgl.mean_nodes(g, "h")

Define a funciton to use the model to evaluate unseen data:

In [ ]:
#model inference function
def evaluate(graph, device, model):
    model.eval()

    with torch.no_grad():

        graph = graph.to(device)
        feat = graph.ndata.pop("attr")

        #edge weights -if any-
        e_weights=graph.edata['weight']

        logits = model(graph, feat,e_weights)
        #print(logits)
        _, predicted = torch.max(logits, 1)  #argmax

    return predicted

This definition rebuilds the graph from data received from Grasshopper through Hops (nodes and edges), and uses that to evaluate it  to return a prediction from the pre-trained model:

In [ ]:
#graph classifier
def plan_eval(nodes,edges):
    #1. BUILD THE GRAPH from the nodes and edges
    #print('here')
    #print(nodes[0])
    #print(edges[0])

    #deserialize
    graph_nodes=[]
    for n in nodes:
        graph_nodes.append(json.loads(n))

    graph_edges=[]
    for e in edges:
        graph_edges.append(json.loads(e))

    #Create an edge list for each graph from the edges dataframe
    graph_edge_list=[[graph_edges[i][0], graph_edges[i][1]] for i in range(len(graph_edges))]  #Notice that the incident nodes must be the first two elements of each list

    #Build the Graph
    FPlan_g=nx.Graph()
    FPlan_g.add_nodes_from(range(len(graph_nodes)))
    FPlan_g.add_edges_from(graph_edge_list)



    #2. Add nodes features (create dictionaries)

    #we have 5 node feature, *****Please notice that we are using x,y,z as features in this example, in your case if they are not node features don't add them
    x_dict={}
    y_dict={}
    z_dict={}
    RoomType_dict={}
    RoomQualit_dict={}

    for i,n in enumerate(graph_nodes):
        x_dict[i]=n[0]
        y_dict[i]=n[1]
        z_dict[i]=n[2]
        RoomType_dict[i]=n[3]
        RoomQualit_dict[i]=n[4]

    nx.set_node_attributes(FPlan_g,x_dict,'x')
    nx.set_node_attributes(FPlan_g,y_dict,'y')
    nx.set_node_attributes(FPlan_g,z_dict,'z')
    nx.set_node_attributes(FPlan_g,RoomType_dict ,'RoomType')
    nx.set_node_attributes(FPlan_g,RoomQualit_dict ,'RoomQualit')

    #3. Add edge features - if any- (create dictionaries)

    # #combine the length and angle values as edge weights as we did in the training set
    # graph_edge_weights=[]
    # for i in graph_edges:
    #     length=i[2]
    #     angle=i[3]
    #     g_w=math.tan(math.radians(angle)) * length
    #     graph_edge_weights.append(g_w)

    #print(len(graph_edge_weights))
    graph_edge_weights=[[graph_edges[i][2]] for i in range(len(graph_edges))]

    weight_dic={}
    for i,edge in enumerate(graph_edge_list):
        weight_dic[(edge[0],edge[1])] = graph_edge_weights[i]

    nx.set_edge_attributes(FPlan_g,weight_dic,'weight')

    #4. Convert to a dgl graph and add both the nodes attributes and edges weight
    t_g_dgl=dgl.from_networkx(FPlan_g, node_attrs=['x','y','z','RoomType','RoomQualit'])

    #add edge attribures ('weight') ------------>
    weight_att=nx.get_edge_attributes(FPlan_g,'weight')

    undirected_weight_att={}
    for i,j in zip(weight_att.keys(),weight_att.values()):
        original_key=i
        reverse_key=(i[1],i[0])   #the reverse edge will take same attribute

        undirected_weight_att[original_key] = j
        undirected_weight_att[reverse_key]  = j

    #add the edge weight to dgl
    #build a tensor of the attributes, the tensor must have the same order as dgl graph edges
    src_indicies=t_g_dgl.edges()[0]    #two tensors of source and destination indicies
    dis_indicies=t_g_dgl.edges()[1]
    weight_tensor_for_dgl=[]
    for src_index, dis_index in zip(src_indicies,dis_indicies):
        for k,v in zip(undirected_weight_att.keys() , undirected_weight_att.values() ):
            if k[0]==src_index and k[1]==dis_index:
                weight_tensor_for_dgl.append(v)


    #add edge weight to the dgl graph
    weight_tensor_for_dgl=torch.tensor(weight_tensor_for_dgl)
    t_g_dgl.edata['weight'] = weight_tensor_for_dgl

    #add all node features to one tensor "attr"
    t_g_dgl.ndata["x"]=torch.reshape(t_g_dgl.ndata["x"],(t_g_dgl.ndata["x"].shape[0],1))
    t_g_dgl.ndata["y"]=torch.reshape(t_g_dgl.ndata["y"],(t_g_dgl.ndata["y"].shape[0],1))
    t_g_dgl.ndata["z"]=torch.reshape(t_g_dgl.ndata["z"],(t_g_dgl.ndata["z"].shape[0],1))
    t_g_dgl.ndata["RoomType"]=torch.reshape(t_g_dgl.ndata["RoomType"],(t_g_dgl.ndata["RoomType"].shape[0],1))
    t_g_dgl.ndata["RoomQualit"]=torch.reshape(t_g_dgl.ndata["RoomQualit"],(t_g_dgl.ndata["RoomQualit"].shape[0],1))
    t_g_dgl.ndata["attr"]=torch.cat([t_g_dgl.ndata["RoomType"], t_g_dgl.ndata["RoomQualit"] ],1)
    t_g_dgl.ndata["attr"]=t_g_dgl.ndata["attr"].type(torch.float32)


    #5. Predict Plan quality
    n_features=t_g_dgl.ndata['attr'].shape[1]
    n_hidden=192 #depends on what you used in the model
    n_classes=4


    loaded_model=SAGE(n_features, n_hidden, n_classes)
    loaded_model.load_state_dict(torch.load(model_path))

    device='cpu'
    predicted_class=evaluate(t_g_dgl,device,loaded_model)

    if predicted_class[0]==0:
        class_name='Class D'
    elif predicted_class[0]==1:
        class_name='Class C'
    elif predicted_class[0]==2:
        class_name='Class B'
    elif predicted_class[0]==3:
        class_name='Class A'

    return f'predicted class is {predicted_class[0]} - {class_name}'

## Hops Server

After everything is setup, we spawn a server with an endpoint that we'll use to bridge Python and Grasshopper. The cell will run continiously, it won't stop and continue to serve until you manualy stop it:

In [ ]:
"""Hops default HTTP server example"""
import ghhops_server as hs
import rhino3dm

hops = hs.Hops()

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))


@hops.component(
    "/Plan_Eval",
    name = "Plan_Eval",
    inputs=[
        hs.HopsString('graph_node_list', 'N', 'Nodes of the input graph' ,hs.HopsParamAccess.LIST),
        hs.HopsString('graph_edge_list', 'E', 'Edges of the input graph' ,hs.HopsParamAccess.LIST),

    ],
    outputs=[
        hs.HopsString("class","c","Prediction"),
    ]
)

def Plan_Eval(nodes,edges):

    return plan_eval(nodes,edges)

if __name__ == "__main__":
    hops.start(debug=True)

INFO:Hops:Starting hops python server on localhost:5000


 * ngrok tunnel "https://1276-35-185-42-222.ngrok.io" -> "http://127.0.0.1:5000/"


DEBUG:Hops:Getting component metadata: <HopsComponent /Plan_Eval [graph_node_list,graph_edge_list -> Plan_Eval -> class] >
DEBUG:Hops:True : {"Uri": "/Plan_Eval", "Name": "Plan_Eval", "Nickname": null, "Description": null, "Category": "Hops", "Subcategory": "Hops Python", "Inputs": [{"Name": "graph_node_list", "Nickname": "N", "Description": "Nodes of the input graph", "ParamType": "Text", "ResultType": "System.String", "AtLeast": 1, "AtMost": 2147483647}, {"Name": "graph_edge_list", "Nickname": "E", "Description": "Edges of the input graph", "ParamType": "Text", "ResultType": "System.String", "AtLeast": 1, "AtMost": 2147483647}], "Outputs": [{"Name": "class", "Nickname": "c", "Description": "Prediction", "ParamType": "Text", "ResultType": "System.String", "AtLeast": 1, "AtMost": 1}]}
INFO:Hops:127.0.0.1 - - [20/Jun/2023 13:48:53] "GET /Plan_Eval HTTP/1.1" 200 -
DEBUG:Hops:Getting component metadata: <HopsComponent /Plan_Eval [graph_node_list,graph_edge_list -> Plan_Eval -> class] >
DE

After running the previous cell, you need to get the address of the address of the ngrok tunnel from the output of the cell (looks like `https://ee65-35-185-51-24.ngrok.io` but not exactly). This will change everytime you run the cell, so you need to update it in your gh definition.

Also, you will need  the endpoint that you created `/shade_classifier`. You will need that to spawn the hops component in grasshopper.

The final endpoint that you need for hops looks like this:

`https://ee65-35-185-51-24.ngrok.io/shade_classifier`

### Troubleshooting
In case the server gets stuck (like in the output above), usually after running the server for the second time, you may need to disconnect ngrok and kill the process that is using port 5000:

In [ ]:
ngrok.disconnect(5000)

You can also check the processes using port 5000:

In [ ]:
!sudo lsof -i:5000

COMMAND   PID USER   FD   TYPE  DEVICE SIZE/OFF NODE NAME
python3 40035 root   44u  IPv4 1351572      0t0  TCP localhost:5000 (LISTEN)


In [ ]:
!kill -9 40035  #this number should be the PID number of the process running in port 5000 as seen in the cell above

after killing this process, the runtime has restared, and you should be able to restart the hops server after running all the cells again.  (don't forget to update  the new ngrok address for hops). Enjoy!